In [4]:
!pip3 install torch torchvision matplotlib

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
  Using cached torch-2.0.1-cp38-none-macosx_10_9_x86_64.whl (143.1 MB)
  Using cached torchvision-0.15.2-cp38-cp38-macosx_10_9_x86_64.whl (1.5 MB)
  Using cached matplotlib-3.7.2-cp38-cp38-macosx_10_12_x86_64.whl (7.4 MB)

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip


In [5]:
import cv2
import torch
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import numpy as np

In [7]:
def rotate_coordinates(coords, angle, center):
    # 중심 좌표를 기준으로 회전 각도로 변환
    angle_rad = np.radians(angle)
    cos_theta = np.cos(angle_rad)
    sin_theta = np.sin(angle_rad)

    # 회전 변환 행렬 계산
    rotation_matrix = np.array([[cos_theta, -sin_theta],
                                [sin_theta, cos_theta]])

    # 중심 좌표를 원점으로 변환
    centered_coords = coords - center

    # 회전 변환 적용
    rotated_coords = np.dot(centered_coords, rotation_matrix.T)

    # 원점 좌표로 복원
    restored_coords = rotated_coords + center

    return restored_coords

하나의 bounding box로 테스트

In [8]:
def draw_rotated_bbox(image, coords, angle, color=(255, 0, 0), width=2):
    # PIL 이미지로 변환
    pil_image = Image.fromarray(image)

    # 이미지에 그리기 위한 Draw 객체 생성
    draw = ImageDraw.Draw(pil_image)

    # 좌표 회전
    #rotated_coords = rotate_coordinates(coords, -angle, center=(image.shape[1] // 2, image.shape[0] // 2))
    # 좌표 순회하며 선 그리기
    for i in range(0, 4):
        pt1 = tuple(coords[i])
        pt2 = tuple(coords[(i + 1) % 4])
        #if (pt1[0] < 0 or pt1[0] > 1023 or pt1[0] < 0 or pt1[1] > 1023):
        #  print(pt1)
        #  continue
        #elif (pt2[0] < 0 or pt2[0] > 1023 or pt2[0] < 0 or pt2[1] > 1023):
        #  print(pt2)
        #  continue
        draw.line([pt1, pt2], fill=color, width=width)

    # PIL 이미지를 다시 NumPy 배열로 변환
    result_image = np.array(pil_image)

    return result_image

In [9]:
def rotate_image(image, angle):
    # 이미지의 높이와 너비 추출
    height, width = image.shape[:2]

    # 회전을 위한 변환 행렬 계산
    center = (width // 2, height // 2)
    matrix = cv2.getRotationMatrix2D(center, angle, 1.0)

    # 이미지 회전
    rotated_image = cv2.warpAffine(image, matrix, (width, height))

    return rotated_image


In [10]:
def parse_coordinates(line):
    # 문자열을 공백을 기준으로 분리하여 숫자들로 이루어진 리스트 생성
    coords = line.split()[:8]
    coords = [int(coord) for coord in coords]

    return coords

In [18]:
# 텍스트 파일 경로
file_path = "/Users/jinheo/Desktop/42AI/OBJ00607_PS3_K3_NIA0086.txt"
# 텍스트 파일 열기
with open(file_path, 'r') as file:
    image_path = "/Users/jinheo/Desktop/42AI/OBJ00607_PS3_K3_NIA0086.png"
    angle = 0
    image = cv2.imread(image_path)
    # image = rotate_image(image, angle) #회전 안돌릴거면 angle=0으로 바꾸고  주석 처리
    # 각 라인을 순회하며 좌표 파싱
    for line in file:
        # 좌표 파싱해서 numpy로 만들기
        coords = np.array(parse_coordinates(line))
        coords = coords.reshape(4, 2)
        image = draw_rotated_bbox(image, coords, angle)
cv2.imshow("Window Name", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import os

In [ ]:
def convert_coordinates(coords_list):
    converted_coords_list = []
    for coords in coords_list:
        x1, y1 = coords[0]
        x2, y2 = coords[1]
        x3, y3 = coords[2]
        x4, y4 = coords[3]
        classname = "container"
        difficult = 0

        converted_coords = f"{x1} {y1} {x2} {y2} {x3} {y3} {x4} {y4} {classname} {difficult}"
        converted_coords_list.append(converted_coords)

    return converted_coords_list


In [ ]:
# 입력 경로
image_folder_path = "/content/drive/MyDrive/container_dataset_png/train_images"
label_folder_path = "/content/drive/MyDrive/container_dataset_png/train_labels_dota"

# 출력 경로
output_image_folder_path = "/content/drive/MyDrive/container_dataset_png/train_images_rotate"
output_label_folder_path = "/content/drive/MyDrive/container_dataset_png/train_labels_dota_rotate"
check_path = "/content/drive/MyDrive/container_dataset_png/check_images"

#os.makedirs(output_image_folder_path, exist_ok=True)
os.makedirs(output_label_folder_path, exist_ok=True)
#os.makedirs(check_path, exist_ok=True)


image_files = os.listdir(image_folder_path)

# 이미지와 텍스트 파일 처리 반복문
for image_file in image_files:

  for i in range(0, 151, 30):
    rotate_angle = i
    # 이미지 경로
    image_path = os.path.join(image_folder_path, image_file)

    # 이미지 읽기
    image = cv2.imread(image_path)
    check_image = cv2.imread(image_path)

    # 이미지 회전
    rotated_image = rotate_image(image, rotate_angle)
    check_image = rotate_image(check_image, rotate_angle)

    # 회전된 이미지 파일 이름 정의
    #rotated_image_file = os.path.splitext(image_file)[0] + "_rotate_" + str(i) + '.png'

    # 회전된 이미지 저장
    #output_image_path = os.path.join(output_image_folder_path, rotated_image_file)
    #cv2.imwrite(output_image_path, rotated_image)

    # 텍스트 파일 경로
    label_file = os.path.splitext(image_file)[0] + ".txt"
    label_path = os.path.join(label_folder_path, label_file)

    # 텍스트 파일 열기
    res_coords = []
    with open(label_path, 'r') as file:
        for line in file:
        # 좌표 파싱해서 numpy로 만들기
          coords = np.array(parse_coordinates(line))
          coords = coords.reshape(4, 2)
          coords = rotate_coordinates(coords, -rotate_angle, center=(rotated_image.shape[1] // 2, rotated_image.shape[0] // 2))
          res_coords.append(coords)
          #check_image = draw_rotated_bbox(check_image, coords, rotate_angle)
    converted_coords_list = convert_coordinates(res_coords)
    #print(converted_coords_list)
    #check_image_file = os.path.splitext(image_file)[0] + "_rotate_" + str(i) + "_check" + '.png'
    #check_image_path = os.path.join(check_path, check_image_file)
    #cv2.imwrite(check_image_path, check_image)
    # 회전된 좌표를 파일에 저장
    output_label_file = os.path.splitext(image_file)[0] + "_rotate_" + str(i) + '.txt'
    output_label_path = os.path.join(output_label_folder_path, output_label_file)
    with open(output_label_path, 'w') as file:
      for converted_coords in converted_coords_list:
        file.write(converted_coords + '\n')
        #함수를 통해 구한 res_coords를 적절한 형식에 맞춰서 넣어주면 된다

  # 완료 메시지 출력
  #print("이미지와 텍스트 파일 처리가 완료되었습니다.")

In [ ]:
def count_files_in_directory(directory_path):
    count = 0
    for filename in os.listdir(directory_path):
        if os.path.isfile(os.path.join(directory_path, filename)):
            count += 1
    return count

# 특정 디렉토리 경로를 지정하여 파일 개수를 세는 함수 호출
directory_path = "/content/drive/MyDrive/container_dataset_png/train_images"
file_count = count_files_in_directory(directory_path)
print("이미지 파일 개수:", file_count)
directory_path = "/content/drive/MyDrive/container_dataset_png/train_labels_dota"
file_count = count_files_in_directory(directory_path)
print("라벨 파일 개수:", file_count)

directory_path = "/content/drive/MyDrive/container_dataset_png/train_images_rotate"
file_count = count_files_in_directory(directory_path)
print("증강 이미지 파일 개수:", file_count)
directory_path = "/content/drive/MyDrive/container_dataset_png/train_labels_dota_rotate"
file_count = count_files_in_directory(directory_path)
print("증강 라벨 파일 개수:", file_count)
directory_path = "/content/drive/MyDrive/container_dataset_png/check_images"
file_count = count_files_in_directory(directory_path)
print("체크 이미지 파일 개수:", file_count)